In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import brentq
from scipy.interpolate import interp2d

import warnings
warnings.filterwarnings('ignore')

### Part 1

In [2]:
# Reading Data
OIS_data = pd.read_excel('IR Data.xlsx', sheet_name='OIS')

# Retriving relevant columns
OIS_data = OIS_data[['Tenor','Rate']]

# Changing tenors to numeric results for operations
OIS_data['Tenor'] = OIS_data['Tenor'].str[:-1]

OIS_data['Tenor'] = OIS_data['Tenor'].astype(float)
OIS_data['Rate'] = OIS_data['Rate'].astype(float)

# Manually changing 6m to 0.5y
OIS_data.iloc[0:1, 0] = 0.5

OIS_data

,Tenor,Rate
0,0.5,0.00250
1,1.0,0.00300
2,2.0,0.00325
3,3.0,0.00335
4,4.0,0.00350
5,5.0,0.00360
6,7.0,0.00400
7,10.0,0.00450
8,15.0,0.00500
9,20.0,0.00525


In [3]:
### Calculating D(0,6m) and f0

ois_6m = OIS_data.iloc[0]['Rate']

### Calculating D(0,6m)
d06m = 1 / (1 + (180/360) * ois_6m)
print("d06m:", d06m)

### Calculating f0 f(0,6m)
f0 = (((0.5*ois_6m) +1)**(1/180) - 1)*360
print("f0:", f0)

d06m: 0.9987515605493134
f0: 0.0024984474705913584


In [4]:
### Calculating D(0,1y) and f1
### Use f to calculate D from this point onward, becasue able to get f directly using OIS rate

ois_1y = OIS_data.iloc[1]['Rate']

### Calculating f1 f(6m,1y)
f1 = (((ois_1y+1)/((1+f0/360)**180))**(1/180)-1) * 360
print("f1:", f1)

### Calculating D01y
d01y = 1 / ( (1+f0/360)**180 * (1+f1/360)**180 ) # d01y = 1 / geosum of all compounding factors
print("d01y:", d01y)

f1: 0.0034925961006493367
d01y: 0.9970089730807411


In [5]:
### Iterate through the tenors from 1-30 (expect 31 total entries)
### Follow the procedure as above, calculate the new forward rate using PV_fixed - PV_float = 0
### Once forward rate is calculated, discount factor can be found

### Have OIS_rates

### Keep track of discount_factors +1 every iteration, forward_rates +1 every iteration,

def calculateForward(row_index, forward_rate, oisDiscFactor): # forward_rate is the objective variable

    # Will be calculating the latest discount factor, and the latest forward
    # latest discount factor is reliant on the latest forward => discount the discount factor from the prev period another time with the new forward rate

    # current discount factor (discounting the latest discount(prev) one more time)
    d_current = oisDiscFactor[-1] / ((1+ forward_rate/360)**360)

    # Fixed Leg 
    # = OIS rate * Sum of Discount Factors, which will be the sum of oisDiscFactor which is running and the newest one
    pv_fixed = (sum(oisDiscFactor) + d_current) * OIS_data['Rate'].iloc[row_index]

    # Float Leg
    # - simplification -> (sum of discount factors up till n-1) * OISrate_n-1 + discountfactor_n * (1 + f_n/360)**360
    # ^ write out the simplification
    pv_float = sum(oisDiscFactor) * OIS_data['Rate'].iloc[row_index-1] \
               + d_current * ((1 + forward_rate/360)**360 -1)

    return pv_fixed - pv_float

def calculateForwardNoOis(row_index, forward_rate, num_inbetween, oisDiscFactor):

    # Since there are tenors with missing OIS, we will assume flat OIS rate from the latest OIS rate (i.e. the current one)
    # (e.g) Missing OIS_6, use OIS_7 for D(0,6y)

    d_list = [oisDiscFactor[-1] / (1+ forward_rate/360)**(360 * n) for n in range(1, num_inbetween+1)]
    # if have OIS_5, OIS_8, need to get d_6, d_7, d_8 using OIS_8 
    # => d_6 = d_5 / (1 + f/360)**360
    # => d_7 - d_6 / (1 + f/360)**360 = d_5/ (1 + f/360)**360 ** 2 = d_5 / (1 + f/360)**(360*2)

    # Fixed Leg 
    # = OIS rate * Sum of Discount Factors, which will be the sum of oisDiscFactor which is running and the newest one
    pv_fixed = (sum(oisDiscFactor) + sum(d_list)) * OIS_data['Rate'].iloc[row_index]

    # Float Leg
    # - simplification -> (sum of discount factors up till n-1) * OISrate_n-1 + discountfactor_n * (1 + f_n/360)**360
    pv_float = sum(oisDiscFactor) * OIS_data['Rate'].iloc[row_index-1] \
               + sum(d_list) * ((1 + forward_rate/360)**360 -1)

    return pv_fixed - pv_float

In [6]:
### Initialize oisDiscFactor and forward rates to add to with each iteration

oisDiscFactor = [d01y]
forwardRates = [f1]


### Loooping from index 2 (2y Tenor) to index 10 (30y Tenor) +1 to include last

for index in range(2, 10+1):

    diff = int(OIS_data['Tenor'].iloc[index] - OIS_data['Tenor'].iloc[index-1])

    if diff == 1:
        objective_function = lambda forward_rate: calculateForward(index, forward_rate, oisDiscFactor)
        current_forward_rate = brentq(objective_function, -10, 20)

        # update lists
        oisDiscFactor.append(oisDiscFactor[-1]/((1+current_forward_rate/360)**360))
        forwardRates.append(current_forward_rate)
        # print('oisDiscFactor_test:', oisDiscFactor_te0st)
        # print('forward_rates_test:', forward_rates_test)

    else:
        objective_function = lambda forward_rate: calculateForwardNoOis(index, forward_rate, diff, oisDiscFactor)
        current_forward_rate = brentq(objective_function, -10, 20)

        # update lists
        new_oisDiscFactor = [oisDiscFactor[-1]/((1+current_forward_rate/360)**(360*n)) for n in range(1, diff+1)]
        new_forward_rates = [current_forward_rate for n in range(1, diff+1)]
        oisDiscFactor.extend(new_oisDiscFactor)
        forwardRates.extend(new_forward_rates)

print (oisDiscFactor)
print (forwardRates)

print(len(oisDiscFactor))
print(len(forwardRates))

[0.9970089730807411, 0.9935307459132384, 0.9900151412182555, 0.9861166497151707, 0.9821841197331733, 0.9772827418427382, 0.9724058232209292, 0.9668984862217808, 0.9614223406862148, 0.9559772099576361, 0.9502359248215516, 0.944529119958292, 0.9388565882906638, 0.9332181239851107, 0.9276135224442457, 0.9220401959122448, 0.9165003552748335, 0.910993799340653, 0.9055203281271514, 0.90007974285332, 0.8946702352342599, 0.8892932389264618, 0.8839485585359208, 0.8786359998429557, 0.8733553697951516, 0.8681064765003451, 0.8628891292196506, 0.8577031383605289, 0.8525483154698985, 0.8474244732272864]
[0.0034925961006493367, 0.0034947783831754194, 0.0035447888626619064, 0.003945605189108556, 0.00399589036444839, 0.0050028120052352815, 0.0050028120052352815, 0.005679763843614161, 0.005679763843614161, 0.005679763843614161, 0.006023828527909487, 0.006023828527909487, 0.006023828527909487, 0.006023828527909487, 0.006023828527909487, 0.006026414558321769, 0.006026414558321769, 0.006026414558321769, 0.

In [7]:
# interpolating 0.5 year oisDiscountFactors 

oisDiscFactor_all = [d06m, d01y]

for original_index in range(1,len(oisDiscFactor)):
    currentDiscount = oisDiscFactor_all[-1]
    nextFullYearDiscount = oisDiscFactor[original_index]
    nextHalfYearDiscount = (nextFullYearDiscount + currentDiscount) /2
    oisDiscFactor_all.append(nextHalfYearDiscount)
    oisDiscFactor_all.append(nextFullYearDiscount)

print(len(oisDiscFactor_all))
oisDiscFactor_all

60


[0.9987515605493134,
 0.9970089730807411,
 0.9952698594969898,
 0.9935307459132384,
 0.991772943565747,
 0.9900151412182555,
 0.9880658954667131,
 0.9861166497151707,
 0.9841503847241719,
 0.9821841197331733,
 0.9797334307879557,
 0.9772827418427382,
 0.9748442825318338,
 0.9724058232209292,
 0.969652154721355,
 0.9668984862217808,
 0.9641604134539978,
 0.9614223406862148,
 0.9586997753219255,
 0.9559772099576361,
 0.9531065673895939,
 0.9502359248215516,
 0.9473825223899218,
 0.944529119958292,
 0.9416928541244779,
 0.9388565882906638,
 0.9360373561378872,
 0.9332181239851107,
 0.9304158232146782,
 0.9276135224442457,
 0.9248268591782453,
 0.9220401959122448,
 0.9192702755935391,
 0.9165003552748335,
 0.9137470773077433,
 0.910993799340653,
 0.9082570637339022,
 0.9055203281271514,
 0.9028000354902357,
 0.90007974285332,
 0.89737498904379,
 0.8946702352342599,
 0.8919817370803609,
 0.8892932389264618,
 0.8866208987311913,
 0.8839485585359208,
 0.8812922791894382,
 0.8786359998429557,


In [8]:
oisDiscFactor_all
# add D(0,0) for reference consistency
oisDiscFactor_all = [1] + oisDiscFactor_all
print(len(oisDiscFactor_all))
oisDiscFactor_all

61


[1,
 0.9987515605493134,
 0.9970089730807411,
 0.9952698594969898,
 0.9935307459132384,
 0.991772943565747,
 0.9900151412182555,
 0.9880658954667131,
 0.9861166497151707,
 0.9841503847241719,
 0.9821841197331733,
 0.9797334307879557,
 0.9772827418427382,
 0.9748442825318338,
 0.9724058232209292,
 0.969652154721355,
 0.9668984862217808,
 0.9641604134539978,
 0.9614223406862148,
 0.9586997753219255,
 0.9559772099576361,
 0.9531065673895939,
 0.9502359248215516,
 0.9473825223899218,
 0.944529119958292,
 0.9416928541244779,
 0.9388565882906638,
 0.9360373561378872,
 0.9332181239851107,
 0.9304158232146782,
 0.9276135224442457,
 0.9248268591782453,
 0.9220401959122448,
 0.9192702755935391,
 0.9165003552748335,
 0.9137470773077433,
 0.910993799340653,
 0.9082570637339022,
 0.9055203281271514,
 0.9028000354902357,
 0.90007974285332,
 0.89737498904379,
 0.8946702352342599,
 0.8919817370803609,
 0.8892932389264618,
 0.8866208987311913,
 0.8839485585359208,
 0.8812922791894382,
 0.87863599984295

In [9]:
IRS_data = pd.read_excel('IR Data.xlsx', sheet_name='IRS')

# Retriving relevant columns
IRS_data = IRS_data[['Tenor','Rate']]

# Changing tenors to numeric results for operations
IRS_data['Tenor'] = IRS_data['Tenor'].str[:-1]

IRS_data['Tenor'] = IRS_data['Tenor'].astype(float)
IRS_data['Rate'] = IRS_data['Rate'].astype(float)

# Manually changing 6m to 0.5y
IRS_data.iloc[0, 0] = 0.5

IRS_data

,Tenor,Rate
0,0.5,0.0250
1,1.0,0.0280
2,2.0,0.0300
3,3.0,0.0315
4,4.0,0.0325
5,5.0,0.0330
6,7.0,0.0350
7,10.0,0.0370
8,15.0,0.0400
9,20.0,0.0450


In [10]:
def calculateLiborDisc(oisDiscFactor, liborFwdRates, fixedRate, prevDiscount, discountFactor, numYear): # objective variable = discountFactor
    pv_fixed = 0.5 * sum(oisDiscFactor[1:int(numYear)*2+1]) * fixedRate # fixed leg, sum of discount factors * fixed rate
    # day-count = 0.5 because semi annual
    # sum of discount factors in increments of 0.5, starting with 0.5y => for year 2: 2*2+1 = 5 => sum(6m, 1y, 1.5y, 2y)
    # * fixed rate (IRS rate)
    pv_float = 0 # (d(0,6m) * libor(0, 6m) + d(0,1y) * libor(6m, 1y) + d(0, 1.5y) * libor(1y, 1.5y)) * 0.5 
    limit = len(liborFwdRates)-2 # -2 because for y3, you would only have libfwd rates for [(0,0), (0,6m), (6m,1y), (1y,1.5y), (1.5y, 2y)]
    tempDiff = (prevDiscount - discountFactor) / 2 # difference between discountfactor we calculating now and the latest discount factor by linear interpolation rules
    tempPrev = prevDiscount
    for index in range(int(numYear) *  2):
        if index <= limit:
            pv_float += 0.5 * oisDiscFactor[index+1] * liborFwdRates[index+1] # within limit means summing up til the prev period, e.g. now calculate y3, your limit for forward rates is up til y2
        else:
            pv_float += (tempPrev - (tempPrev - tempDiff)) / (tempPrev - tempDiff) * oisDiscFactor[index+1]
            # D(0, 2.5) = D(0,2) - ( D(0,2) - D(0, 2.5) ) / D(0, 2.5)
            # D(0, 2.5) = (D(0,3) + D(0,2)) / 2 
            # tempPrev D(0, 2) - diff(D(0,2) and D(0,2.5) = D(0, 2.5) 
            tempPrev = prevDiscount-tempDiff # update now tempPrev is D(0, 2.5) for the calculation of D(0, 3)
    return pv_fixed - pv_float

def calculateLiborDiscWithoutIRS(oisDiscFactor, liborFwdRates, numHalfYear, fixedRate, prevDiscount, discountFactor, numYear): # objective variable = discountFactor
    pv_fixed = 0.5 * sum(oisDiscFactor[1:int(numYear)*2+1]) * fixedRate
    # same as with IRS
    pv_float = 0
    tempDiff = (prevDiscount - discountFactor) / numHalfYear
    # temp diff is different now, 
    # numHalfYear = 4 => 5.5, 6, 6.5, 7(dis)
    # linear interpolation from D(0,5) to D(0,7), each increase will be D(0,7) - D(0,5) / 4
    tempPrev = prevDiscount
    limit = len(liborFwdRates)-2
    for index in range( int(numYear) * 2): #loop is the same since all all the needed D()s will be in terms of dis just minus a different amt of interpolation factor
        if index <= limit:
            pv_float += 0.5 * oisDiscFactor[index+1] * liborFwdRates[index+1]
        else:
            test = 2* oisDiscFactor[index+1] *(tempPrev-(tempPrev - tempDiff))/(tempPrev - tempDiff)
            pv_float +=  0.5 * test
            tempPrev = tempPrev-tempDiff

    return pv_fixed - pv_float

def calculateLiborFwd(prevDiscount, discountFactor):
    temp = 2 * (prevDiscount - discountFactor) / discountFactor
    return temp 


In [11]:
### initialise liborDiscFactors and liborForwards

liborDiscFactor_all = [1] #D(0,0)
liborForwardRates = [0, 0.025] #L(0,0), L(0,6m)

In [12]:
### Manually Calculating D_L(0,6m), D_L(0,1y)

# D_L(0, 6m)
libor_d06m = 1 / (1 + (0.5 * IRS_data['Rate'][0]))
liborDiscFactor_all.append(libor_d06m)

#D_L(0, 1y) using brent

def libor_d01y_objectiveFunction(libor_d01y):
    pv_fixed = 0.5 * IRS_data['Rate'][1] * sum(oisDiscFactor_all[1:3])
    pv_float = 0.5 * IRS_data['Rate'][0] * oisDiscFactor_all[1] + \
               0.5 * oisDiscFactor_all[2] * (2*(libor_d06m - libor_d01y)/libor_d01y)
    return pv_fixed - pv_float

libor_d01y = brentq(lambda x: libor_d01y_objectiveFunction(x), 1e-12, 1)
liborDiscFactor_all.append(libor_d01y)

# Adding L(6m, 1y)
L6m1y = 2*(libor_d06m - libor_d01y) / libor_d01y
liborForwardRates.append(L6m1y)

In [13]:
print(liborDiscFactor_all)
print(liborForwardRates)

[1, 0.9876543209876544, 0.9725768302912439]
[0, 0.025, 0.031005243445693578]


In [14]:
dataCounter = 2
tempPrevSumFloat = 0.0
tempPrevDiscount = 1.0

index = 2
while index < 31:
    tempYear = IRS_data['Tenor'][dataCounter]

    while index > tempYear:
        dataCounter += 1
        tempYear = IRS_data['Tenor'][dataCounter]

    # current fixed rate for this year
    tempFixedRate = IRS_data['Rate'][dataCounter]

    # most recent discount factor (last period)
    prevDiscount = liborDiscFactor_all[-1]
    tempDiff = 0

    if ( index != tempYear): # no irs rates
        numHalfYear = int((tempYear - index + 1)*2)
        tempDiscount = brentq(lambda x: calculateLiborDiscWithoutIRS(oisDiscFactor_all,
                                                                     liborForwardRates,
                                                                     numHalfYear,
                                                                     tempFixedRate,
                                                                     prevDiscount,
                                                                     x,
                                                                     tempYear),
                              0.0001, 1)
        tempDiff = (prevDiscount - tempDiscount) / numHalfYear
        for i in range(numHalfYear):
            prevDiscount = liborDiscFactor_all[-1]
            tempDiscount = prevDiscount - tempDiff
            liborDiscFactor_all.append(tempDiscount)

            tempLiborFwd = calculateLiborFwd(prevDiscount, tempDiscount)
            liborForwardRates.append(tempLiborFwd)

        index = tempYear+1 # advance the loop by 1 year


    else: # with irs
        tempDiscount = brentq(lambda x: calculateLiborDisc(oisDiscFactor_all,
                                                           liborForwardRates,
                                                           tempFixedRate,
                                                           prevDiscount,
                                                           x,
                                                           tempYear),
                              0.0001, 1)

        tempDiscountMinusHalf = (prevDiscount + tempDiscount) / 2 # linear interpolation for between last period and current period

        liborDiscFactor_all.append(tempDiscountMinusHalf)
        liborDiscFactor_all.append(tempDiscount)

        tempLiborFwdMinusHalf = calculateLiborFwd(prevDiscount, tempDiscountMinusHalf)
        tempLiborFwd = calculateLiborFwd(tempDiscountMinusHalf, tempDiscount)

        liborForwardRates.append(tempLiborFwdMinusHalf)
        liborForwardRates.append(tempLiborFwd)
        index += 1 # advance the loop
        

In [15]:
print(len(liborDiscFactor_all))
liborDiscFactor_all

61


[1,
 0.9876543209876544,
 0.9725768302912439,
 0.9573779178550954,
 0.942179005418947,
 0.9263302686640764,
 0.9104815319092058,
 0.8947311053593985,
 0.8789806788095911,
 0.8639849837996798,
 0.8489892887897685,
 0.8327960030675967,
 0.8166027173454249,
 0.8004094316232531,
 0.7842161459010812,
 0.7689651425085141,
 0.753714139115947,
 0.7384631357233798,
 0.7232121323308127,
 0.7079611289382456,
 0.6927101255456785,
 0.6785470704606977,
 0.664384015375717,
 0.6502209602907363,
 0.6360579052057556,
 0.6218948501207748,
 0.6077317950357941,
 0.5935687399508134,
 0.5794056848658327,
 0.5652426297808519,
 0.5510795746958712,
 0.5367931788694701,
 0.522506783043069,
 0.5082203872166678,
 0.4939339913902667,
 0.4796475955638656,
 0.4653611997374645,
 0.45107480391106336,
 0.43678840808466224,
 0.4225020122582611,
 0.40821561643186,
 0.3989876618104544,
 0.38975970718904884,
 0.38053175256764327,
 0.3713037979462377,
 0.3620758433248321,
 0.35284788870342654,
 0.34361993408202096,
 0.334391

In [16]:
def calculate_forward_swap_rate(start_year, tenor, ois_discount, liborForwards):
    start = start_year
    end = start_year + tenor

    # denominator is the "fixed" = sum of ois_discount factors * daycount 
    ois_disc_factors = ois_discount[start*2+1 : end*2+1]
    # indexes are 2*year+1 due to the construction of oisDiscFactors_all and that year's discount factor is not taken into account 
    # => [d00, d06m, d01y, d01.5y, d02y, 02.5y ...] 
    # year1, first discount factor is d01.5y => 2*1+1 = 3
    # year2, first discount factor is d02.5y => 2*2+1 = 5 

    denominator = sum(ois_disc_factors) * 0.5 # 0.5 is day fraction

    counter = start * 2 + 1 # start from the index of the first payment
    # numerator is the "float" leg
    numerator = 0
    for index in range(tenor*2): # no need +1 because D(0, start) is not included
        numerator += ois_discount[counter] * liborForwards[counter] * 0.5
        counter += 1 # advance to the next D
    return numerator/denominator
    

In [17]:
forwardSwaps = [(1,1),
                (1,2),
                (1,3),
                (1,5),
                (1,10),
                (5,1),
                (5,2),
                (5,3),
                (5,5),
                (5,10),
                (10,1),
                (10,2),
                (10,3),
                (10,5),
                (10,10)]

fwdSwapRatesDF = pd.DataFrame(columns=["Start","Tenor","liborForwardSwapRate"])
rows_list = []

for swap_combination in forwardSwaps:
    swapStart = swap_combination[0]
    swapTenor = swap_combination[1]
    swapRate = calculate_forward_swap_rate(swapStart, swapTenor, oisDiscFactor_all, liborForwardRates)
    newRow = pd.DataFrame({'Start':[str(swapStart)+"Y"], 'Tenor':[str(swapTenor)+"Y"], 'liborForwardSwapRate':[swapRate]})
    rows_list.append(newRow)

fwdSwapRatesDF = pd.concat(rows_list, ignore_index=True)

### All Data

In [18]:
T0 = [0 for i in range(0,61)]
TN = [0.5*i for i in range(0,61)]

oisDiscFactorsDF = pd.DataFrame({'T0': T0, 'TN': TN, 'DiscountFactor': oisDiscFactor_all})
oisDiscFactorsDF

,T0,TN,DiscountFactor
0,0,0.0,1.000000
1,0,0.5,0.998752
2,0,1.0,0.997009
3,0,1.5,0.995270
4,0,2.0,0.993531
...,...,...,...
56,0,28.0,0.857703
57,0,28.5,0.855126
58,0,29.0,0.852548
59,0,29.5,0.849986


In [19]:
len(liborDiscFactor_all)

61

In [20]:
T0 = [0 for i in range(0,61)]
TN = [0.5*i for i in range(0,61)]

liborDiscFactorsDF = pd.DataFrame({'T0': T0, 'TN': TN, 'DiscountFactor': liborDiscFactor_all})
liborDiscFactorsDF

,T0,TN,DiscountFactor
0,0,0.0,1.000000
1,0,0.5,0.987654
2,0,1.0,0.972577
3,0,1.5,0.957378
4,0,2.0,0.942179
...,...,...,...
56,0,28.0,0.260568
57,0,28.5,0.251340
58,0,29.0,0.242112
59,0,29.5,0.232884


In [21]:
fwdSwapRatesDF

,Start,Tenor,liborForwardSwapRate
0,1Y,1Y,0.032007
1,1Y,2Y,0.033259
2,1Y,3Y,0.034011
3,1Y,5Y,0.035255
4,1Y,10Y,0.038428
5,5Y,1Y,0.039274
6,5Y,2Y,0.040075
7,5Y,3Y,0.040072
8,5Y,5Y,0.041093
9,5Y,10Y,0.043634


In [23]:
with pd.ExcelWriter("Data_1_Output.xlsx") as writer:
    # Write each DataFrame to a separate sheet
    oisDiscFactorsDF.to_excel(writer, sheet_name='oisDiscFactors', index=False)
    liborDiscFactorsDF.to_excel(writer, sheet_name='liborDiscFactors', index=False)
    fwdSwapRatesDF.to_excel(writer, sheet_name='fwdSwapRates', index=False)